<a href="https://colab.research.google.com/github/ACM-Research/NLP-Summarizer/blob/master/Notebooks/t5-summarizer/t5_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

code originally gotten from
https://towardsdatascience.com/simple-abstractive-text-summarization-with-pretrained-t5-text-to-text-transfer-transformer-10f6d602c426

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import os,sys
import json
import spacy
from collections import Counter

#T5 model Types
#
#t5-small
#t5-base
#t5-large
#t5-3B
#t5-11B

model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

nlp = spacy.load("en_core_web_sm")
# Configures model to use GPU
device = torch.device('cuda')
model.to(torch.device("cuda:0"))

print('download complete')


download complete


In [ ]:
path = "/content/drive/My Drive/sotu"
dirs = os.listdir(path)

In [ ]:
# run when doing for first time
os.mkdir("/content/drive/My Drive/sotuSummaries","")

In [ ]:
  summaries = open("/content/drive/My Drive/sotuSummaries/super_summary.txt","w")
for file in dirs:
    reader = open("/content/drive/My Drive/sotu/"+file,"r")

    text = reader.read()


    # Adjust text stripper to make sure text is stripped properly
    preprocess_text = text.strip().replace("\n"," ")
    t5_prepared_Text = "summarize: "+preprocess_text
    print ("original text preprocessed: \n", preprocess_text)

    # Adjust max_length for speech size based on tokens
    tokenized_text = tokenizer.encode(t5_prepared_Text, max_length = 2700,truncation=True, return_tensors="pt").to(device)


    # summmarize 
    # Change no_repeat_ngram_size to adjust size of Ngrams in summary
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=4,
                                        min_length=30,
                                        max_length=100,
                                        early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


    print ("\n\nSummarized text: \n",output)
    print()

    original_text_pos = nlp(preprocess_text)
    summary_pos = nlp(output)

    print()

    jsonStuff = {"original_text:": preprocess_text, 
                    "summary_text": output,
                    "compression_ratio": str(len(output)/len(preprocess_text)),
                    "original_pos_tags": Counter([token.pos_ for token in original_text_pos]),
                    "summary_pos_tags": Counter([token.pos_ for token in summary_pos])
                    }
        
    summaries.write(json.dumps(jsonStuff) + "\n")
  
  reader.close()
  summaries.close()

  # Compression Ratio
  # print (len(output)/len(text))

